# 2019 NBA Regular Season Evaluation

## Import needed packages and config

In [186]:
import requests
import mysql.connector
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup 


## Connect to MySql  and create database. Commented out once DB is created

In [43]:
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
)
cursor = cnx.cursor(buffered=True)

## (re)Connect to database, with database defined, to create table

In [34]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

## Make API Call to grab 2019 NBA reg season data


In [283]:

# headers = {'api-nba-v1.p.rapidapi.com' '{}'.format(api_key)}
headers = nba_headers

In [297]:
headers = nba_headers

nba_data = []

response = requests.get(url, headers=headers)
data = response.json()
nba_data.append(data)

In [301]:
nba_data = nba_data[0]['api']['games']

In [314]:
nba_data

[{'seasonYear': '2018',
  'league': 'utah',
  'gameId': '5706',
  'startTimeUTC': '2018-07-02T23:00:00.000Z',
  'endTimeUTC': '2018-07-03T00:48:00.000Z',
  'arena': 'Vivint Smart Home Arena',
  'city': 'Salt Lake City',
  'country': 'USA',
  'clock': '',
  'gameDuration': '1:44',
  'currentPeriod': '4/4',
  'halftime': '0',
  'EndOfPeriod': '0',
  'seasonStage': '2',
  'statusShortGame': '3',
  'statusGame': 'Finished',
  'vTeam': {'teamId': '1',
   'shortName': 'ATL',
   'fullName': 'Atlanta Hawks',
   'nickName': 'Hawks',
   'logo': 'https://upload.wikimedia.org/wikipedia/fr/e/ee/Hawks_2016.png',
   'score': {'points': '88'}},
  'hTeam': {'teamId': '19',
   'shortName': 'MEM',
   'fullName': 'Memphis Grizzlies',
   'nickName': 'Grizzlies',
   'logo': 'https://upload.wikimedia.org/wikipedia/en/thumb/f/f1/Memphis_Grizzlies.svg/1200px-Memphis_Grizzlies.svg.png',
   'score': {'points': '103'}}},
 {'seasonYear': '2018',
  'league': 'utah',
  'gameId': '5707',
  'startTimeUTC': '2018-07-03

In [312]:
nba_data[1]['vTeam']['shortName']

'SAS'

In [337]:
parsed_games_nba = []
for game in nba_data:
    parsed_game = {}
    parsed_game['game_id'] = game['gameId']
    parsed_game['month'] = game['startTimeUTC'][5:7]
    parsed_game['away_team'] = game['vTeam']['shortName']
    parsed_game['away_team_score'] = game['vTeam']['score']['points']
    parsed_game['home_team'] = game['hTeam']['shortName']
    parsed_game['home_team_score'] = game['hTeam']['score']['points']
    parsed_games_nba.append(parsed_game)

In [369]:
parsed_games_nba

[{'game_id': '5706',
  'month': '07',
  'away_team': 'ATL',
  'away_team_score': 88,
  'home_team': 'MEM',
  'home_team_score': 103},
 {'game_id': '5707',
  'month': '07',
  'away_team': 'SAS',
  'away_team_score': 76,
  'home_team': 'UTA',
  'home_team_score': 92},
 {'game_id': '5618',
  'month': '07',
  'away_team': 'MIA',
  'away_team_score': 68,
  'home_team': 'GSW',
  'home_team_score': 79},
 {'game_id': '5619',
  'month': '07',
  'away_team': 'LAL',
  'away_team_score': 93,
  'home_team': 'SAC',
  'home_team_score': 98},
 {'game_id': '5708',
  'month': '07',
  'away_team': 'SAS',
  'away_team_score': 103,
  'home_team': 'ATL',
  'home_team_score': 81},
 {'game_id': '5709',
  'month': '07',
  'away_team': 'MEM',
  'away_team_score': 95,
  'home_team': 'UTA',
  'home_team_score': 92},
 {'game_id': '5620',
  'month': '07',
  'away_team': 'MIA',
  'away_team_score': 89,
  'home_team': 'LAL',
  'home_team_score': 74},
 {'game_id': '5621',
  'month': '07',
  'away_team': 'GSW',
  'away

In [359]:
parsed_games_nba[0]['away_team_score'] = int(parsed_games_nba[0]['away_team_score'])

In [361]:
print(type(parsed_games_nba[0]['away_team_score']))

<class 'int'>


In [368]:
for i in range(len(parsed_games_nba)):
    parsed_games_nba[i]['home_team_score'] = int(parsed_games_nba[i]['home_team_score'])
#     parsed_game['home_team_score'] = game['hTeam']['score']['points']
#     parsed_games_nba.append(parsed_game)

ValueError: invalid literal for int() with base 10: ''

## Creating NHL Table

In [326]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [ ]:
# cursor.execute("""
# CREATE DATABASE MLB
# """)

# cursor.close()
# cnx.close()

In [338]:
cursor.execute("""
    CREATE TABLE nba (
    game_id varchar(11) NOT NULL primary key,
    month varchar(2) not null,
    away_team varchar(11) not null,
    away_team_score int(3) not null,
    home_team varchar(11) not null,
    home_team_score int(3) not null
    )
""")

ProgrammingError: 1050 (42S01): Table 'nba' already exists

## Inserting to table

In [370]:
nba_games_to_insert = []

for game in parsed_games_nba:

    nba_game_to_insert = (
        game['game_id'], 
        game['month'], 
        game['away_team'],
        game['away_team_score'],
        game['home_team'], 
        game['home_team_score']
    )
    
    nba_games_to_insert.append(nba_game_to_insert)

In [371]:
print(type(nba_games_to_insert))

<class 'list'>


In [373]:
cursor.execute("""TRUNCATE TABLE nba""")

In [374]:
insert_query = """
    INSERT INTO nba (
            game_id,
            month,
            away_team,
            away_team_score,
            home_team,
            home_team_score
        )
    VALUES (
            %s,
            %s,
            %s,
            %s,
            %s,
            %s
            )
"""
cursor.executemany(insert_query, nba_games_to_insert)
cnx.commit()

### Questions (Null Hypothesis)

Is there a "home field" advantage(in general , 2018 season)?
    
Will a home team preform better (by month)?

If home team cover more than away team ("good teams win, great teams cover")?


## Creating MLB DataFrame

In [23]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [25]:
cursor.execute(  """SELECT * FROM nba
                    order by game_id;""")
#assigning database (pointed in MySql) as a dataframe using pandas                         
df_nba = pd.DataFrame(cursor.fetchall())


#assigning the column names 
df_nba.columns=['game_id','month', 'away_team', 'away_team_score', 'home_team', 'home_team_score', 'away_team_moneyline', 'home_team_moneyline', 'spread']

In [26]:
df_nba

,game_id,month,away_team,away_team_score,home_team,home_team_score,away_team_moneyline,home_team_moneyline,spread
0,13093,10,PHI,9,OTT,5,242,-265,2.0
1,13094,10,VEG,3,WAS,7,-242,219,-1.7
2,13095,10,ARI,4,ANA,3,-236,216,-1.7
3,13096,10,MON,3,TOR,4,-144,132,-1.7
4,13097,10,BOS,0,WAS,9,-306,283,-1.7
5,13098,10,CGY,3,VAN,7,242,-265,2.0
6,13099,10,ANA,7,SJ,3,-196,184,-1.7
7,13100,10,BOS,5,BUF,0,230,-253,2.0
8,13101,10,NYI,3,CAR,1,-219,201,-1.7
9,13102,10,NAS,4,NYR,3,213,-230,2.0
